In [5]:
import cv2
import ultralytics
from ultralytics import YOLO
import torch

print('ultralistic version:', ultralytics.__version__)
print('torch version:', torch.__version__)

ultralistic version: 8.3.62
torch version: 2.5.1


In [6]:
def detect_objects_in_video(video_path, output_video):
    yolo_model = YOLO('./runs/detect/train8/weights/best.pt') 
    video_capture = cv2.VideoCapture(video_path)
    width = int(video_capture.get(3))
    height = int(video_capture.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    result_video_path = output_video + '.avi'
    out = cv2.VideoWriter(result_video_path, fourcc, 20.0, (width, height))
    wearpon = False
    wearpon_list = {}

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        results = yolo_model(frame, verbose=False)

        for result in results:
            classes = result.names
            cls = result.boxes.cls
            conf = result.boxes.conf
            detections = result.boxes.xyxy

            for pos, detection in enumerate(detections):
                if conf[pos] >= 0.5:
                    xmin, ymin, xmax, ymax = detection
                    label = f"{classes[int(cls[pos])]}" 
                    color = (0, int(cls[pos]), 255)
                    cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
                    cv2.putText(frame, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
                    wearpon = ('gun' in label) or ('knife' in label) or ('knives' in label)
                    if wearpon:
                        if label not in wearpon_list:
                            wearpon_list[label] = 1
                        else:
                            wearpon_list[label] += 1                        

        out.write(frame)
    video_capture.release()
    out.release()

    return wearpon_list

In [7]:
wearpon_list = detect_objects_in_video('./input/video.mp4', './res_video_1')

In [8]:
if len(wearpon_list) > 0:
  print('Wearpon detected!!!')
  for key in wearpon_list.keys():
    print(f' - Wearpon {key}: Times: {wearpon_list[key]}')

#result_video_path = detect_objects_in_video('./input/video2.mp4', './res_video_2')
#print(f"Detected objects video saved at: {result_video_path}")

Wearpon detected!!!
 - Wearpon knife: Times: 81
